<a href="https://colab.research.google.com/github/attila5287/scrap_and_dataviz/blob/main/states_ue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
!pip install matplotlib pandas requests folium geopandas

In [187]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
import json
import requests
import folium
from folium.plugins import TimeSliderChoropleth
import geopandas as gpd

In [188]:
urljson = 'https://raw.githubusercontent.com/shawnbot/'\
+'topogram/master/data/us-states.geojson'

In [189]:
urljson = 'https://raw.githubusercontent.com/python-visualization/'\
+'folium/main/examples/data/us-states.json'  

In [190]:
gdf = gpd.read_file( requests.get(urljson).text )
gdf = gdf[[ 'name', 'geometry' ]] \
        .sort_values('name')
len(gdf)
gdf.head()

,name,geometry
0,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
2,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."
3,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496..."
4,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."


In [191]:
gdf['state'] = [
       name.replace(' ', '') for name in gdf['name']
]


In [192]:
# gdf

In [193]:
url = 'https://www.ncsl.org/research/labor-and-employment/'\
+'state-unemployment-update.aspx'

In [194]:
raw_df = pd.read_html(url)[0]

In [195]:
# raw_df

In [196]:
for month in raw_df.transpose().columns:
  pass
  # print( raw_df.set_index('State') )

In [197]:
l = []
for i,state in enumerate(raw_df['State']):
    pass
    d={}
    for j, month in enumerate( raw_df.columns[1:] ):
        pass
        d= {
            'State' : state,
            'Month' : month.replace('.', ''),
            # 'Date'  : dt_index[ j ],  
            'Rate' :  list(
                raw_df.loc[ raw_df['State'] == state ][month])[0],
        }
        l.append(d)
        
df = pd.DataFrame(l)    

In [198]:
df.loc[df['State'] == 'D.C.', 'State'] = 'District of Columbia'

In [199]:
df['name'] = [
              state.replace(' ', '') for state in df['State']
]

In [200]:
# df['name'].unique()

In [201]:
nan_states = list(df.loc[
        df['Rate'].isnull()
 ]['State'])
nan_states

['New Hampshire', 'New Jersey', 'Ohio']

In [202]:
avg = { 
    state: round(df.loc[ df['State'] == state ].mean()['Rate'], 2)\
            for state in nan_states
}
avg_list = [ round(df.loc[ df['State'] == state ].mean()['Rate'], 2)\
            for state in nan_states ]
# df.loc[ df['Rate'].isnull()]['Rate'] = avg
print(df.loc[ df['Rate'].isnull()]['Rate'] )

# df.transpose().fillna(value=avg, inplace=True)
nan_index = df.loc[ df['Rate'].isnull()]['Rate'].index 
for index, avg in zip(nan_index,avg_list):
  pass
  # print(df['Rate'][index])
  # df['Rate'][index] = avg
  # print(df['Rate'][index])
  
  if index>1:
    pass
    df['Rate'][index] = df['Rate'][index-1]
  else:
    pass
    df['Rate'][index] = df['Rate'][index+1]


print(df.loc[ df['Rate'].isnull()]['Rate'] )


356   NaN
368   NaN
428   NaN
Name: Rate, dtype: float64
Series([], Name: Rate, dtype: float64)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [204]:
maxrate = df.max()['Rate']
minrate = df.min()['Rate']

In [205]:
df['scaled'] = [
  (rate-minrate)/(maxrate-minrate) for rate in df['Rate']
]

In [252]:
from branca.colormap import linear
cmap = linear.BuPu_06.scale( minrate, maxrate)
cmap

In [242]:
df['opacity'] = 0.77

In [243]:
df['color'] = df['Rate']
df[ 'color' ] = df[ 'color' ].apply(cmap)
# df['color'].unique()

In [244]:
datetime_index = pd.date_range("2021-1-1", periods=12, freq="M")
dt_index_epochs = datetime_index.astype(int)
dt_index = dt_index_epochs.astype("U10")
# dt_index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


In [245]:
d = {}

for i,state in enumerate(gdf['state']):
  pass
  v = df.loc[ df['name'] == state ][[ 'color', 'opacity']]\
  .set_index(dt_index)\
  .to_dict(orient="index")
  d[state] = v

# d['NorthDakota']
# d['Colorado']

In [264]:
import folium
from folium.plugins import TimeSliderChoropleth

m = folium.Map([36, -99], tiles="stamentoner", zoom_start=4)

g = TimeSliderChoropleth(
    gdf.set_index('state').to_json(),
    styledict=d,
    ).add_to(m)
#specify the min and max values of your data
colormap = cmap
colormap.caption = 'State Unemployment Rate'
colormap.add_to(m)
m

In [247]:
# folium.map.CustomPane("labels").add_to(m)

# # Final layer associated to custom pane via the appropriate kwarg
# folium.TileLayer("stamentonerlabels", pane="labels").add_to(m)
